In [1]:
import numpy as np
import pandas as pd
from glob import glob

In [2]:
election_type = "councilors" # councilors

output = []
for path in glob(f"2021/{election_type}/*.xlsx"):
    print(path)
    xlsx = pd.ExcelFile(path)

    if election_type == "councilors":
        for sheet in xlsx.sheet_names:
            frag = pd.read_excel(xlsx, sheet_name=sheet)
            frag = frag.rename(columns={"SubPacto": "Sub Pacto", "Subpacto": "Sub Pacto", "Región Nº": "Nº Región", "Nº Región": "Nº Región"})
            output.append(frag)
    else:
        frag = pd.read_excel(xlsx, sheet_name="Votación", header=6)
        frag["Candidato"] = frag["Nombres"] + " " + frag["Primer apellido"] + " " + frag["Segundo apellido"]
        frag = frag.rename(columns={"Votos": "Votos TER"})
        output.append(frag)

df = pd.concat(output, ignore_index=True)
df.head()

2021/councilors\01_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\02_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\03_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\04_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\05_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\06_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\07_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\08_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\09_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\10_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\11_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\12_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\13_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\14_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\15_Resultados_Mesa_Concejales_TER.xlsx
2021/councilors\16_Resultados_Mesa_Concejales_TER.xlsx


,Nº Región,Región,Provincia,CS,Distrito,Comuna,Circunscripción,Mesa Nº,Tipo,Fusionadas,...,Nº en Voto,Pacto,Sub Pacto,Partido,Candidato,Votos TER,N° Región,Nº Mesa,N° mesa,MesaNº
0,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,300.0,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,KARL HEINZ HARDER LLANOS,6.0,NaN,NaN,NaN,NaN
1,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,301.0,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,HUMBERTO SADY GALAZ ROJAS,1.0,NaN,NaN,NaN,NaN
2,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,302.0,CHILE DIGNO VERDE Y SOBERANO,PARTIDO FEDERACION REGIONALISTA VERDE SOCIAL E...,FEDERACION REGIONALISTA VERDE SOCIAL,GIGLIOLA ANGELICA INOSTROZA ESCARATE,4.0,NaN,NaN,NaN,NaN
3,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,303.0,CHILE DIGNO VERDE Y SOBERANO,PARTIDO FEDERACION REGIONALISTA VERDE SOCIAL E...,FEDERACION REGIONALISTA VERDE SOCIAL,ESTER YAZMIN CISTERNAS VERA,0.0,NaN,NaN,NaN,NaN
4,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,1.0,M,1M,...,304.0,CHILE DIGNO VERDE Y SOBERANO,PARTIDO FEDERACION REGIONALISTA VERDE SOCIAL E...,FEDERACION REGIONALISTA VERDE SOCIAL,GISELLE DEL CARMEN MUÑOZ GONZALEZ,4.0,NaN,NaN,NaN,NaN


In [3]:
list(df)

['Nº Región',
 'Región',
 'Provincia',
 'CS',
 'Distrito',
 'Comuna',
 'Circunscripción',
 'Mesa Nº',
 'Tipo',
 'Fusionadas',
 'Lista',
 'Nº en Voto',
 'Pacto',
 'Sub Pacto',
 'Partido',
 'Candidato',
 'Votos TER',
 'N° Región',
 'Nº Mesa',
 'N°  mesa',
 'MesaNº']

In [4]:
df = df.rename(columns={"Circunscripción electoral": "Circunscripción", "Mesa": "Mesa Nº"})

In [5]:
list(df)

['Nº Región',
 'Región',
 'Provincia',
 'CS',
 'Distrito',
 'Comuna',
 'Circunscripción',
 'Mesa Nº',
 'Tipo',
 'Fusionadas',
 'Lista',
 'Nº en Voto',
 'Pacto',
 'Sub Pacto',
 'Partido',
 'Candidato',
 'Votos TER',
 'N° Región',
 'Nº Mesa',
 'N°  mesa',
 'MesaNº']

In [6]:
df[df["Comuna"] == "CONCEPCION"]

,Nº Región,Región,Provincia,CS,Distrito,Comuna,Circunscripción,Mesa Nº,Tipo,Fusionadas,...,Nº en Voto,Pacto,Sub Pacto,Partido,Candidato,Votos TER,N° Región,Nº Mesa,N° mesa,MesaNº
968222,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,ANDALIEN,NaN,NaN,89,...,300.0,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,ALEX PATRICIO ITURRA JARA,1.0,NaN,NaN,89.0,NaN
968223,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,ANDALIEN,NaN,NaN,89,...,301.0,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,CAROLINA MAGDALENA REBOLLEDO REBOLLEDO,2.0,NaN,NaN,89.0,NaN
968224,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,ANDALIEN,NaN,NaN,89,...,302.0,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,JOEL ANTONIO ARROYO PAREDES,1.0,NaN,NaN,89.0,NaN
968225,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,ANDALIEN,NaN,NaN,89,...,303.0,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,OLIMPIA FERNANDA RIVEROS RAVELO,2.0,NaN,NaN,89.0,NaN
968226,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,ANDALIEN,NaN,NaN,89,...,304.0,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,PABLO EMILIO BLASET GARRIDO,0.0,NaN,NaN,89.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028095,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,CONCEPCION CENTRO,NaN,V,193V,...,389.0,RADICALES E INDEPENDIENTES,NaN,INDEPENDIENTES,MONICA PAOLA GAYOSO SALGADO,1.0,NaN,NaN,193.0,NaN
1028096,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,CONCEPCION CENTRO,NaN,V,193V,...,390.0,RADICALES E INDEPENDIENTES,NaN,INDEPENDIENTES,GABRIEL PATRICIO GONZALEZ CALDERON,1.0,NaN,NaN,193.0,NaN
1028097,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,CONCEPCION CENTRO,NaN,V,193V,...,391.0,RADICALES E INDEPENDIENTES,NaN,INDEPENDIENTES,PEDRO ANTONIO SAEZ ESPINOZA,1.0,NaN,NaN,193.0,NaN
1028098,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,CONCEPCION CENTRO,NaN,V,193V,...,900.0,NaN,NaN,NaN,VOTOS NULOS,0.0,NaN,NaN,193.0,NaN


In [7]:
df["Votos TER"] = df["Votos TER"].astype(str).str.strip().replace("", 0).astype(float)

In [8]:
df[df.Candidato.fillna("#").str.contains("NULO")].Candidato.unique()

array(['VOTOS NULOS'], dtype=object)

In [9]:
def create_polling_id(data, columns=[]):
    output = data[columns[0]].fillna("###").copy()

    for column in columns[1:]:
        output += "-" + data[column].fillna("###").astype(str)
    return output

cols = ["Región", "Distrito", "Comuna", "Circunscripción", "Mesa Nº"]
if election_type == "councilors":
    cols += ["Tipo", "Fusionadas"]
df["polling_id"] = create_polling_id(df, columns=cols)

In [10]:
df[df["Candidato"] == "ITALO GIOVANNI POLIZZI BUSTOS"]

,Nº Región,Región,Provincia,CS,Distrito,Comuna,Circunscripción,Mesa Nº,Tipo,Fusionadas,...,Pacto,Sub Pacto,Partido,Candidato,Votos TER,N° Región,Nº Mesa,N° mesa,MesaNº,polling_id
968269,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,ANDALIEN,NaN,NaN,89,...,DIGNIDAD AHORA,IGUALDAD PARA CHILE,INDEPENDIENTES,ITALO GIOVANNI POLIZZI BUSTOS,3.0,NaN,NaN,89.0,NaN,DEL BIOBIO-20.0-CONCEPCION-ANDALIEN-###-###-89
968363,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,ANDALIEN,NaN,M,1M,...,DIGNIDAD AHORA,IGUALDAD PARA CHILE,INDEPENDIENTES,ITALO GIOVANNI POLIZZI BUSTOS,1.0,NaN,NaN,1.0,NaN,DEL BIOBIO-20.0-CONCEPCION-ANDALIEN-###-M-1M
968457,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,ANDALIEN,NaN,M,2M,...,DIGNIDAD AHORA,IGUALDAD PARA CHILE,INDEPENDIENTES,ITALO GIOVANNI POLIZZI BUSTOS,2.0,NaN,NaN,2.0,NaN,DEL BIOBIO-20.0-CONCEPCION-ANDALIEN-###-M-2M
968551,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,ANDALIEN,NaN,M,3M,...,DIGNIDAD AHORA,IGUALDAD PARA CHILE,INDEPENDIENTES,ITALO GIOVANNI POLIZZI BUSTOS,3.0,NaN,NaN,3.0,NaN,DEL BIOBIO-20.0-CONCEPCION-ANDALIEN-###-M-3M
968645,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,ANDALIEN,NaN,M,4M,...,DIGNIDAD AHORA,IGUALDAD PARA CHILE,INDEPENDIENTES,ITALO GIOVANNI POLIZZI BUSTOS,1.0,NaN,NaN,4.0,NaN,DEL BIOBIO-20.0-CONCEPCION-ANDALIEN-###-M-4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027677,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,CONCEPCION CENTRO,NaN,V,189V,...,DIGNIDAD AHORA,IGUALDAD PARA CHILE,INDEPENDIENTES,ITALO GIOVANNI POLIZZI BUSTOS,5.0,NaN,NaN,189.0,NaN,DEL BIOBIO-20.0-CONCEPCION-CONCEPCION CENTRO-#...
1027771,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,CONCEPCION CENTRO,NaN,V,190V,...,DIGNIDAD AHORA,IGUALDAD PARA CHILE,INDEPENDIENTES,ITALO GIOVANNI POLIZZI BUSTOS,8.0,NaN,NaN,190.0,NaN,DEL BIOBIO-20.0-CONCEPCION-CONCEPCION CENTRO-#...
1027865,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,CONCEPCION CENTRO,NaN,V,191V,...,DIGNIDAD AHORA,IGUALDAD PARA CHILE,INDEPENDIENTES,ITALO GIOVANNI POLIZZI BUSTOS,10.0,NaN,NaN,191.0,NaN,DEL BIOBIO-20.0-CONCEPCION-CONCEPCION CENTRO-#...
1027959,8.0,DEL BIOBIO,CONCEPCION,10.0,20.0,CONCEPCION,CONCEPCION CENTRO,NaN,V,192V,...,DIGNIDAD AHORA,IGUALDAD PARA CHILE,INDEPENDIENTES,ITALO GIOVANNI POLIZZI BUSTOS,2.0,NaN,NaN,192.0,NaN,DEL BIOBIO-20.0-CONCEPCION-CONCEPCION CENTRO-#...


In [11]:
# VOTOS NULOS
# VOTOS EN BLANCO	
# TOTALES
df["Candidato"] = df["Candidato"].replace({
    "VOTOS EN BLANCO": "BLANK",
    "VOTOS EN BLANCOS": "BLANK",
    "VOTOS BLANCOS": "BLANK",
    "VOTOS NULOS": "NULL"
})

df_spoilt = df[df["Candidato"].isin(["NULL", "BLANK"])].copy()
df_spoilt = df_spoilt[["Candidato", "Votos TER", "polling_id"]].rename(columns={"Candidato": "candidate", "Votos TER": "value"})
df_spoilt["flag_candidates"] = 0
df_spoilt.head()

df = df[~df["Candidato"].isin(["BLANK", "NULL", "TOTALES", np.nan])].copy()


In [12]:
df["Sub Pacto"] = df["Sub Pacto"].fillna(df["Pacto"])
data = df[["polling_id", "Candidato", "Partido", "Pacto", "Sub Pacto", "Votos TER"]].dropna(subset=["Pacto"])
data = data.rename(columns={"Pacto": "coalition", "Sub Pacto": "subcoalition", "Candidato": "candidate", "Partido": "party", "Votos TER": "value"})
data["value"] = data["value"].astype(str).apply(lambda x: x.strip()).apply(lambda x: 0 if x == "" else x).astype(float)

tt = data.groupby(["polling_id", "coalition", "subcoalition", "party", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum()).fillna(0)
tt = tt.reset_index()
tt["flag_candidates"] = 1

tt.head()

,polling_id,coalition,subcoalition,party,candidate,value,rate,flag_candidates
0,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,ALFREDO ENRIQUE TAPIA CONTRERAS,2.0,0.017241,1
1,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,DOLORES PAZ CAUTIVO AHUMADA,6.0,0.051724,1
2,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,NINOSKA ANGELICA GONZALEZ FLORES,3.0,0.025862,1
3,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,PABLO LOLET PONCE PONCE,1.0,0.008621,1
4,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,SCARLETH TAMARA FERNANDEZ PINTO,2.0,0.017241,1


In [13]:
data = pd.concat([
    tt,
    df_spoilt
])

In [14]:
list(df)

['Nº Región',
 'Región',
 'Provincia',
 'CS',
 'Distrito',
 'Comuna',
 'Circunscripción',
 'Mesa Nº',
 'Tipo',
 'Fusionadas',
 'Lista',
 'Nº en Voto',
 'Pacto',
 'Sub Pacto',
 'Partido',
 'Candidato',
 'Votos TER',
 'N° Región',
 'Nº Mesa',
 'N°  mesa',
 'MesaNº',
 'polling_id']

In [15]:
df_location = df[["polling_id", "Región", "Distrito", "Comuna", "Circunscripción", "Mesa Nº"]].drop_duplicates()
df_location = df_location.rename(columns={
    "Región": "region",
    "Comuna": "commune",
    "Distrito": "district",
    "Circunscripción": "circunscription",
    "Local": "place",
    "Mesa Nº": "polling_station"
})

In [16]:
data.head()

,polling_id,coalition,subcoalition,party,candidate,value,rate,flag_candidates
0,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,ALFREDO ENRIQUE TAPIA CONTRERAS,2.0,0.017241,1
1,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,DOLORES PAZ CAUTIVO AHUMADA,6.0,0.051724,1
2,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,NINOSKA ANGELICA GONZALEZ FLORES,3.0,0.025862,1
3,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,PABLO LOLET PONCE PONCE,1.0,0.008621,1
4,ARICA Y PARINACOTA-1.0-ARICA-ARICA NORTE-###-#...,CHILE DIGNO VERDE Y SOBERANO,PARTIDO COMUNISTA E INDEPENDIENTES,PARTIDO COMUNISTA DE CHILE,SCARLETH TAMARA FERNANDEZ PINTO,2.0,0.017241,1


In [17]:
data.to_csv(f"../../data_output/Chile/2021_councilor_polling_station.csv.gz")
df_location.to_csv(f"../../data_output/Chile/2021_councilor_polling_station_location.csv.gz")